<a href="https://colab.research.google.com/github/mjag7682/CS9-1-NLP-for-Twitter-Data-for-predicting-stocks/blob/master/PreprocessingFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install, Import, Mount statements

In [1]:
!pip install contractions
!pip install emoji
!pip install ekphrasis

import pandas as pd
import re
import emoji
import nltk
import contractions
import torch

nltk.download('punkt')
nltk.download('stopwords')
# from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer 
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize 
from nltk.tokenize.treebank import TreebankWordDetokenizer
import requests 
from pprint import pprint
import numpy as np
from nltk.stem import PorterStemmer

import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras
# import bert
# from bert import run_classifier
# from bert import optimization
# from bert import tokenization
import os
import pprint
import json
import random
import string
import sys
from ekphrasis.classes.segmenter import Segmenter
import itertools


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load file

In [3]:
df_comb = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Capstone/Labelled (Percentage change)/WMT_label2.1.csv')
combine_ds = df_comb.sample(frac=1)
combine_ds

,message,label
10779,$WMT how are you bearish fools doing?,1
7731,$WMT more pain for stubborn shorts. -signed st...,1
3159,$WMT layoffs and cost cutting measures coming ...,-1
561,fyi: Huge (over 1 million) call vol in $T and ...,1
12165,The 6 Most Shorted Dow Stocks\n\n$MSFT $CSCO ...,0
...,...,...
7153,Yesterdays Sell On Close Closing Order Imbal...,-1
7790,Why XPO Logistics Inc Stock Is More Than a Spe...,1
10965,"Newport Wealth Strategies Inc.,has filed Form ...",1
12201,$WMT Walmart needs to show us some action today,1


## Preprocess data

In [4]:
combine_ds['message'] = combine_ds['message'].str.lower()
message = combine_ds['message'].tolist()

print(message[:10])

['$wmt how are you bearish fools doing?', '$wmt more pain for stubborn shorts. -signed stubborn long', '$wmt layoffs and cost cutting measures coming -brazil is a disaster.no position. if it gets to 45, maybe..lol', 'fyi: huge (over 1 million) call vol in $t and $vz itm july calls yesterday. chain: http://stk.ly/owk4r5 , http://stk.ly/mqiyye &gt;  what&#39;s up?', 'the 6 most shorted dow stocks\n\n$msft  $csco $vz $ko $v \n\nhttps://247wallst.com/investing/2019/12/26/6-most-shorted-dow-stocks-microsoft-is-king-of-the-hill-in-december/1/', '02:57:30 pm manual exit. closing my $wmt position of 672 shares that was opened oct 29 for a 0.07% gain.', '@sspencer_smb tmobile/sprint still a foreign owed enterprise, either way..  buy american $t and $vz ..', '@scarborory agree .... $s owns the most spectrum in the us comparing w $t , $vz and $tmus', '$baba just had some positive pr on fox news radio. talked about how $baba is destroying $wmt due to it&#39;s success.', '$wmt $ostk $msft employees

In [5]:
def remove_stopwords(msg):
    filtered_sentence = [w for w in msg_tokens if not w in stop_words]
    return filtered_sentence

def remove_punctuation_re(x):
    x = ' '.join(re.sub("https?://\S+","",x).split())     #Removing URLs

    x = ' '.join(re.sub("^@\S+|\s@\S+","",x).split())     #Removing Mentions

    # x = ' '.join(re.sub(r'[^$\w\s]'," ",x).split())
    x = ' '.join(re.sub(r'[^\w\s]'," ",x).split())        #Removes Hashtags

    x = ' '.join(re.sub(r'_'," ",x).split())              #Removing _ from emojis text

    return x

# replace repeating letter
def rpt_replace(match):
    # print(match.group(1))
    return match.group(1)+match.group(1)

# substitute original word with replaced word, if any
def processRepeatings(data):
    re_t= re.sub(message_rpt, rpt_replace, data )
    # print(re_t)
    return re_t

In [6]:
stop_words = sw.words("english")
tweet_tokenizer = TweetTokenizer()
detokenizer = TreebankWordDetokenizer()
# message_p = []

# for repeating characters in words
message_rpt = re.compile(r"(.)\1{2,}", re.IGNORECASE)

# segmenter using the word statistics from Twitter
seg_tw = Segmenter(corpus="twitter")


Reading twitter - 1grams ...
Reading twitter - 2grams ...


In [7]:
message_p = []
for msg in message:
    # remove emojis
    msg = emoji.demojize(msg)

    # fix contractions
    msg = contractions.fix(msg)

    # remove punctuations
    msg = remove_punctuation_re(msg) 

    #tokenize
    msg_tokens = tweet_tokenizer.tokenize(msg)

    #For Hashtags elongated words using Word segmenter
    message_seg = []
    for w in msg_tokens:
      message_seg.append(seg_tw.segment(w))

    # remove stopwords
    msg = remove_stopwords(message_seg)

    if 'rt' in msg:
      # remove retweets
      message_p.append('-1')
    else: 
      # detokenize
      msg = detokenizer.detokenize(msg)

      # removing repeating characters like hurrrryyyyyy-- worrks on tokenized list
      msg = processRepeatings(msg)

      message_p.append(msg)


In [8]:
message_p[:10]

['wmt bearish fools',
 'wmt pain stubborn shorts signed stubborn long',
 'wmt layoffs cost cutting measures coming brazil disaster position gets 45 maybe lol',
 'fyi huge 1 million call vol vz itm july calls yesterday chain gt 39',
 '6 shorted dow stocks msft csco vz ko v',
 '02 57 30 pm manual exit closing wmt position 672 shares opened oct 29 0 07 gain',
 'tmobile sprint still foreign owed enterprise either way buy american vz',
 'agree owns spectrum us comparing w vz tmus',
 'baba positive pr fox news radio talked baba destroying wmt due 39 success',
 'wmt ostk msft employees chant bathroom']

In [10]:
combine_ds['message'] = message_p
combine_ds

,message,label
10779,wmt bearish fools,1
7731,wmt pain stubborn shorts signed stubborn long,1
3159,wmt layoffs cost cutting measures coming brazi...,-1
561,fyi huge 1 million call vol vz itm july calls ...,1
12165,6 shorted dow stocks msft csco vz ko v,0
...,...,...
7153,yesterdays sell close closing order imbalance ...,-1
7790,xpo logistics inc stock speculative buy xpo tg...,1
10965,newport wealth strategies inc filed form 13f q...,1
12201,wmt walmart needs show us action today,1


In [12]:
# Drop Retweets
combine_ds.drop(combine_ds[combine_ds['message'] == '-1'].index, inplace = True) 
combine_ds

,message,label
10779,wmt bearish fools,1
7731,wmt pain stubborn shorts signed stubborn long,1
3159,wmt layoffs cost cutting measures coming brazi...,-1
561,fyi huge 1 million call vol vz itm july calls ...,1
12165,6 shorted dow stocks msft csco vz ko v,0
...,...,...
7153,yesterdays sell close closing order imbalance ...,-1
7790,xpo logistics inc stock speculative buy xpo tg...,1
10965,newport wealth strategies inc filed form 13f q...,1
12201,wmt walmart needs show us action today,1
